In [ ]:
# Data manipulation
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ML Models 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier

# Model Metrics
from sklearn.metrics import accuracy_score

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Set pyplot style
plt.style.use('seaborn')

# Dimensionality Reduction
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE

In [ ]:
# Import dataset from datasets directory
ansur = pd.read_csv('~/DevSpace/Data-Snippets/MachineLearning/datasets/ansur.csv')
ansur.head()

In [ ]:
# Spot perfect correlation in data
cmap = sns.diverging_palette(h_neg=10, h_pos=240, as_cmap=True)
sns.heatmap(ansur.corr(), center=0, cmap=cmap, linewidths=1, annot=True, fmt='.2f')
plt.title('Correlation Matrix for Ansur Dataset')
plt.show()

In [ ]:
# Brute force removal of hightly correlated features
abs_corr_matrix = ansur.corr().abs()

# Upper triagule of corr matrix to false
mask = np.triu(np.ones_like(abs_corr_matrix, dtype=bool))
tri_df = abs_corr_matrix.mask(mask)

# Filter highly correlated dimensions
to_drop = [c for c in tri_df.columns if any(tri_df[c] >  0.95)]

# Drop highly correlated dimensions
reduced_df = ansur.drop(to_drop, axis=1)

In [ ]:
# Check visualization
sns.pairplot(reduced_df, hue='Gender', diag_kind='hist')
plt.show()

In [ ]:
# High dimensional data import
body_dims = pd.read_csv('~/DevSpace/Data-Snippets/MachineLearning/datasets/body_measurements.csv')
body_dims.drop('ID', axis=1, inplace=True)
print(f'Number of dimensions (cols): {len(body_dims.columns)}')

In [ ]:
# Instanciate TSNE Model
tsne_model = TSNE(learning_rate=50)

# Fit and transform numeric data from 4th column
tsne_features = tsne_model.fit_transform(body_dims.iloc[:, 4:])

# Add components to df
body_dims['x'] = tsne_features[:, 0]
body_dims['y'] = tsne_features[:, 1]

# Plot transformed data
sns.scatterplot(x='x', y='y', hue='Gender', data=body_dims)
plt.xlabel('X Dimension')
plt.ylabel('Y Dimension')
plt.title('t-SNE of Body Measurements')
plt.show()

In [ ]:
# Predict 'Gender' based on all body measurements
y = body_dims['Gender']
X = body_dims.drop(['Branch', 'Component', 'Gender', 'x', 'y'], axis=1)

# Split train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=12)

# Instanciate model and fit model
svm = SVC()
svm.fit(X_train, y_train)

# Measure accuracy
acc_train = accuracy_score(y_train, svm.predict(X_train))
acc_test = accuracy_score(y_test, svm.predict(X_test))

# Measure overfitting abs(train - test / test)
print(f'{acc_train = }')
print(f'{acc_test = }')


In [ ]:
# Predict Gender based on 'neckcircumferencebase' feature 
y = body_dims['Gender']
X = body_dims[['neckcircumferencebase']]

# Split train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=12)

# Instanciate model and fit model
svm = SVC()
svm.fit(X_train, y_train)

# Measure accuracy
acc_train = accuracy_score(y_train, svm.predict(X_train))
acc_test = accuracy_score(y_test, svm.predict(X_test))

# Measure overfitting abs(train - test / test)
print(f'{acc_train = }')
print(f'{acc_test = }')

In [ ]:
# Study the distribution of the variances from all dimensions from body_dims

# Normalize columns for better visualization
numeric_dims = body_dims.iloc[:, 3:-2] / body_dims.iloc[:, 3:-2].mean()

# Calculate variances, median and mean
vars = numeric_dims.std() ** 2
median = round(vars.quantile(), 5)
mean = round(vars.mean(), 5)

# Plot variance distribution and annotate
sns.boxplot(y=vars.values)
plt.annotate(f'Median variance: {median}', (-0.45, 0.022))
plt.annotate(f'Mean variance: {mean}', (-0.45, 0.0195))

plt.xlabel('Variances')
plt.ylabel('Normalized Variance')
plt.title('Normalized variance distribution of dimensions')
plt.show()


In [ ]:
# Reduce dimensionality with a variance threshold (median variance)
variance_selection = VarianceThreshold(threshold=0.00482)

# Fit variance selector
variance_selection.fit(numeric_dims)

# Get variance mask
mask = variance_selection.get_support()

# Reduced body_dims
reduced_body = numeric_dims.loc[:, mask]

# New number of dimensions
print(f'Old dimensions: {len(body_dims.columns)}')
print(f'New dimensions: {len(reduced_body.columns)}')

In [ ]:
# Predict Gender based on reduced dataset
y = body_dims['Gender']
X = reduced_body

# Split train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=12)

# Instanciate model and fit model
svm = SVC(C=0.1)
svm.fit(X_train, y_train)

# Measure accuracy
acc_train = accuracy_score(y_train, svm.predict(X_train))
acc_test = accuracy_score(y_test, svm.predict(X_test))

# Measure overfitting abs(train - test / test)
print(f'{acc_train = }')
print(f'{acc_test = }')

In [ ]:
# Predict Gender with a classifier less prone to overfit (simpler model)
log_reg = LogisticRegression(solver='newton-cg', C=0.1)

# Fit Model
log_reg.fit(X_train, y_train)

# Measure Accuracy
acc_train = accuracy_score(y_train, log_reg.predict(X_train))
acc_test = accuracy_score(y_test, log_reg.predict(X_test))

# Measure overfitting
print(f'{acc_train = }')
print(f'{acc_test = }')

In [ ]:
# Create DataFrame of dimensions and coefficients
lr_coefs = dict(zip(reduced_body.columns, np.abs(log_reg.coef_[0])))

# Sort by most important coefficients
lr_coefs = sorted(lr_coefs.items(), key=lambda x: x[1], reverse=True)
pd.Series(dict(lr_coefs[:5]))

In [ ]:
# Instanciate and fit a Recursive Feature Elimination
rfe = RFE(estimator=log_reg, n_features_to_select=1)
rfe.fit(X_train, y_train)

# Sort ranked features by incresing importance
feature_ranking = dict(zip(X.columns, rfe.ranking_))
sorted_features = dict(sorted(feature_ranking.items(), key=lambda x: x[1], reverse=True))

# Order DataFrame and set X, y
ranking_df = reduced_body[sorted_features.keys()]
X, y = ranking_df, body_dims['Gender']

#### Iterate over ordered columns and test accuracy of Logistic Regression

In [ ]:
# Save accuracy scores
accuracy = []

# Iterate over all models
for i in range(len(sorted_features)):

    # Train Test Split with ordered features
    X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, i:], y, train_size=0.7, random_state=12)

    # Train model with those features
    log_reg.fit(X_train, y_train)

    # Measure accuracy
    train_acc = accuracy_score(y_train, log_reg.predict(X_train))
    test_acc = accuracy_score(y_test, log_reg.predict(X_test))
    accuracy.append([len(X_train.columns), train_acc, test_acc])

#### Plot accuracy scores for every RFE selection

In [ ]:
# Extract accuracy data from experiment
accuracy = np.array(accuracy)
features, train, test = np.split(accuracy, 3, axis=1)
features = features.flatten().astype('int')

# Plot lines of training adn testing accuracy
plt.plot(train, label='Train Score')
plt.plot(test, label='Test Score')
plt.xticks(features[::5], features[::-5])

# Annotate figure
plt.title('Train & Test scores by number of features')
plt.ylabel('Score')
plt.xlabel('Number of Features')

# Figure style and legend
plt.xlim(0, 45)
plt.tight_layout()
plt.legend()
plt.show()

### Dimensionality reduction with Decision Tree Classifiers

In [ ]:
# Import High Dimensional Data
diabetes = pd.read_csv('~/DevSpace/Data-Snippets/MachineLearning/datasets/diabetes.csv', index_col='index')


# Set X and y
X = diabetes.drop(['diagnostic'], axis=1)
y = diabetes['diagnostic']

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

# Feature selection with Tree Based Models
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Measure Accuracy
acc_train = accuracy_score(y_train, rf.predict(X_train))
acc_test = accuracy_score(y_test, rf.predict(X_test))

# Sort important features
features = pd.Series(data=rf.feature_importances_.round(2), index=X.columns)
print(features.sort_values(ascending=False), '\n')

# Measure accuracy
print(f'{acc_train = }')
print(f'{acc_test = }')

### Dimensionality reduction with Lasso Regression

In [ ]:
# Set X and y
X = diabetes.drop(['diagnostic', 'bmi'], axis=1)
y = diabetes['bmi']

# Train Test Split and scale data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

# Create the Lasso model and fit to data
la = Lasso(alpha=0.05)
la.fit(X_train_std, y_train)

# Transform the test set with the pre-fitted scaler
X_test_std = scaler.transform(X_test)

# Calculate the coefficient of determination (R squared) on X_test_std
r_squared = la.score(X_test_std, y_test)
print(f"Model predicted {r_squared:.2%} of the variance.")

# Create a list that has True values when coefficients equal 0
zero_coef = la.coef_ == 0

# Calculate how many features have a zero coefficient
n_ignored = sum(zero_coef)
print(f"The model has ignored {n_ignored} out of {len(la.coef_)} features.")

In [ ]:
from sklearn.linear_model import LassoCV
# Create and fit the LassoCV model on the training set
lcv = LassoCV()
lcv.fit(X_train, y_train)
print('Optimal alpha = {0:.3f}'.format(lcv.alpha_))

# Calculate R squared on the test set
r_squared = lcv.score(X_test, y_test)
print(f'The model explains {r_squared:.1%} of the test set variance')

# Create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print(f'{sum(lcv_mask)} features out of {len(lcv_mask)} selected')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Select 10 features with RFE on a GradientBoostingRegressor, drop 3 features on each step
rfe_gb = RFE(estimator=GradientBoostingRegressor(), 
             n_features_to_select=10, step=3, verbose=1)
rfe_gb.fit(X_train, y_train)

# Calculate the R squared on the test set
r_squared = rfe_gb.score(X_test, y_test)
print(f'The model can explain {r_squared:.1%} of the variance in the test set')

# Assign the support array to gb_mask
gb_mask = rfe_gb.support_

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Select 10 features with RFE on a RandomForestRegressor, drop 3 features on each step
rfe_rf = RFE(estimator=RandomForestRegressor(), 
             n_features_to_select=10, step=3, verbose=1)
rfe_rf.fit(X_train, y_train)

# Calculate the R squared on the test set
r_squared = rfe_rf.score(X_test, y_test)
print('The model can explain {0:.1%} of the variance in the test set'.format(r_squared))

# Assign the support array to gb_mask
rf_mask = rfe_rf.support_

In [ ]:
# Sum the votes of the three models
votes = np.sum([lcv_mask, rf_mask, gb_mask], axis=0)

# Create a mask for features selected by all 3 models
meta_mask = votes >= 3

# Apply the dimensionality reduction on X
X_reduced = X.loc[:, meta_mask]

In [ ]:
# Import Linear Model
from sklearn.linear_model import LinearRegression

# Create model
lm = LinearRegression()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=0)

# Plug the reduced dataset into a linear regression pipeline
lm.fit(scaler.fit_transform(X_train), y_train)

# Measure accucary R^2
r_squared = lm.score(scaler.transform(X_test), y_test)

# Result
print('The model can explain {0:.1%} of the variance in the test set using {1:} features.'.format(r_squared, len(lm.coef_)))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Import dataset
ansur = pd.read_csv('~/DevSpace/Data-Snippets/MachineLearning/datasets/ansur.csv')
ansur_df = ansur.drop(['Gender', 'Branch', 'Component', 'ID'], axis=1)

# Create the scaler
scaler = StandardScaler()
ansur_std = scaler.fit_transform(ansur_df)
n_components = 10

# Create the PCA instance and fit and transform the data with pca
pca = PCA(n_components=n_components)
pc = pca.fit_transform(ansur_std)
pc_df = pd.DataFrame(pc, columns=[f'PC_{x}' for x in range(1, n_components + 1)])

In [ ]:
# Create a pairplot of the first n principal components
sns.pairplot(pc_df.iloc[:, :4])
plt.show()

In [ ]:
# Plot variance explanation by component
plt.plot(range(n_components), pca.explained_variance_ratio_.cumsum())

# Annotate plot
plt.title('Explained variance by PCA')
plt.xlabel('Total components')
plt.ylabel('Total variance explained')
plt.show()
